In [ ]:
import math
import torch
from torch import nn

# Model

In [ ]:
class Swish(nn.Module):
  def forward(self, x):
    return x * torch.sigmoid(x)

In [ ]:
class FeatureEx1d(nn.Module):
  def __init__(self, input_size):
    super().__init__()
    self.layers = nn.Sequential(
        nn.linear(input_size, 256),
        nn.functional.relu(),
        nn.linear(256, 128),
        nn.functional.relu(),
        nn.linear(128, 64),
        nn.functional.relu()
    )

  def forward(self, x):
    return self.layers(x)

In [ ]:
class SEblock(nn.Module):
  def __init__(self, ch_in, ch_sq):
    super().__init__()
    self.se = nn.Sequential(
        nn.AdaptiveAvgPool2d(1),
        nn.Conv2d(ch_in, ch_sq, 1),
        Swish(),
        nn.Conv2d(ch_sq, ch_in, 1),
    )
    self.se.apply(weight_init)

  def forward(self, x):
    return x * torch.sigmoid(self.se(x))

def weight_init(m):
  if isinstance(m, nn.Conv2d):
    nn.init.kaiming_normal_(m.weight)

  if isinstance(m, nn.Linear):
    nn.init.kaiming_uniform_(m.weight)
    nn.init.zeros_(m.bias)

In [ ]:
class ConvBN(nn.Module):
  def __init__(self, ch_in, ch_out, kernel_size, stride=1, padding=0, groups=1):
    super().__init__()
    self.layers=nn.Sequential(
        nn.Conv2d(ch_in, ch_out, kernel_size, stride, padding, groups=groups, bias=False),
        nn.BatchNorm2d(ch_out),
    )
    self.layers.apply(weight_init)

  def forward(self, x):
    return self.layers(x)

In [ ]:
class DropConnect(nn.Module):
  def __init__(self, drop_rate):
    super().__init__()
    self.drop_rate=drop_rate

  def forward(self, x):
    if self.training:
      keep_rate=1.0-self.drop_rate
      r = torch.rand([x.size(0),1,1,1], dtype=x.dtype).to(x.device)
      r+=keep_rate
      mask=r.floor()
      return x.div(keep_rate)*mask
    else:
      return x

In [ ]:
class BMConvBlock(nn.Module):
  def __init__(self,ch_in,ch_out,expand_ratio,stride,kernel_size,reduction_ratio=4,drop_connect_rate=0.2):
    super().__init__()
    self.use_residual = (ch_in==ch_out) & (stride==1)
    ch_med = int(ch_in*expand_ratio)
    ch_sq  = max(1, ch_in//reduction_ratio)

    if expand_ratio != 1.0:
      layers = [ConvBN(ch_in, ch_med, 1), Swish()]
    else:
      layers = []
    
    layers.extend([ConvBN(ch_med, ch_med, kernel_size, stride=stride, padding=(kernel_size)//2, groups=ch_med),
                  Swish(),
                  SEblock(ch_med, ch_sq),
                  ConvBN(ch_med, ch_out, 1),
                  ])
    
    if self.use_residual:
      self.drop_connect = DropConnect(drop_connect_rate)

    self.layers = nn.Sequential(*layers)

  def forward(self, x):
    if self.use_residual:
      return x + self.drop_connect(self.layers(x))
    else:
      return self.layers(x)

In [ ]:
class Flatten(nn.Module):
  def forward(self, x):
    return x.view(x.shape[0], -1)

In [ ]:
class MultiModalNet(nn.Module):
  def __init__(self,
               width_mult=1.0,
               depth_mult=1.0,
               resolution=None,
               dropout_rate=0.2,
               num_1d_features=10,
               num_classes=4,
               input_ch=3):
    super().__init__()

    # expand_ratio, channel, repeats, stride, kernel_size                   
    settings = [
        [1,  16, 1, 1, 3],  # MBConv1_3x3, SE, 112 -> 112                   
        [6,  24, 2, 2, 3],  # MBConv6_3x3, SE, 112 ->  56                   
        [6,  40, 2, 2, 5],  # MBConv6_5x5, SE,  56 ->  28                   
        [6,  80, 3, 2, 3],  # MBConv6_3x3, SE,  28 ->  14                   
        [6, 112, 3, 1, 5],  # MBConv6_5x5, SE,  14 ->  14                   
        [6, 192, 4, 2, 5],  # MBConv6_5x5, SE,  14 ->   7                   
        [6, 320, 1, 1, 3]   # MBConv6_3x3, SE,   7 ->   7]                  
    ]
    ch_out = int(math.ceil(32*width_mult))
    features = [nn.AdaptiveAvgPool2d(resolution)] if resolution else []
    features.extend([ConvBN(input_ch, ch_out, 3, stride=2), Swish()])     # -> [32, 3, 3, 3]

    ch_in = ch_out
    for t, c, n, s, k in settings:
      ch_out = int(math.ceil(c*width_mult))
      repeats = int(math.ceil(n*depth_mult))
      for i in range(repeats):
        stride = s if i==0 else 1
        features.extend([BMConvBlock(ch_in, ch_out, t, stride, k)])
        ch_in = ch_out

    ch_last = int(math.ceil(1280*width_mult))
    features.extend([ConvBN(ch_in, ch_last, 1), Swish()])

    self.features2d = nn.Sequential(*features)
    self.reshape = nn.Sequential(
        nn.AdaptiveAvgPool2d(1),
        Flatten(),
        nn.Dropout(dropout_rate),
        nn.Linear(ch_last, 128)
    )
    self.classifier = nn.Sequential(
        nn.Linear(128*2, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, num_classes),
        nn.Sigmoid()
    )

    self.features1d = nn.Sequential(
        nn.Linear(num_1d_features, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
    )

  def forward(self, x):
    x1=self.features2d(x[1])
    x1=self.reshape(x1)
    x0=self.features1d(x[0])
    x = torch.cat((x0,x1), dim=1)
    x=self.classifier(x)

    return x

# Dataset

In [ ]:
import shutil
import os
RESEARCH_WORK_PATH = "/content/drive/MyDrive/Colab Notebooks/BachelorResearch/"


data_dir = "/content/datas/"
if os.path.exists(data_dir):
  shutil.rmtree(data_dir)
for dirs in os.listdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/"):
  if "Q" in dirs:
    shutil.copytree(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + dirs, data_dir + dirs)

In [ ]:
import os
def make_filepath_list(root, train_rate=0.8):
  train_file_list = []
  valid_file_list = []

  for dirs in os.listdir(root):
    if "Q" in dirs:
      file_dir = os.path.join(root, dirs)
      file_list = os.listdir(file_dir)

      num_data = len(file_list)
      num_split = int(num_data * train_rate)

      train_file_list += [os.path.join(root, dirs, file).replace('\\', '/') for file in file_list[:num_split]]
      valid_file_list += [os.path.join(root, dirs, file).replace('\\', '/') for file in file_list[num_split:]]
  
  return train_file_list, valid_file_list


In [ ]:
from torch.utils import data
import numpy as np
import librosa
import pickle
import torch
import cv2

class musicDataset(data.Dataset):

  def __init__(self, file_list, classes, phase='train'):
    self.file_list = file_list
    self.classes = classes
    self.phase = phase
    self.scaler = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/BachelorResearch/models/1d_scaler.pickle', "rb"))

  def __len__(self):
    return len(self.file_list)

  def min_max(self, x): # do min_max normalization ?->https://atmarkit.itmedia.co.jp/ait/articles/2110/07/news027.html
    min = 0
    max = 154.28702 
    # max = 137.74599
    x = np.abs(x)
    result = (x-min)/(max-min)
    return result
    

  def __getitem__(self, index):
    song_path = self.file_list[index]
    y, sr = librosa.load(song_path)
    
    n_fft = 512
    hop_length = 256
    stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    stft = self.min_max(stft)
    stft = np.expand_dims(stft, axis=2)
    stft = torch.from_numpy(stft)
    stft = stft.permute(2,1,0)


    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    # rmse = librosa.feature.rmse(y=y)
    rmse = librosa.feature.rms(y=y) # librosaのバージョンが0.7.0以上の場合はこちらを実行
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    f1d = [np.mean(chroma_stft), np.mean(rmse), np.mean(spec_cent), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)]    
    for e in mfcc:
      f1d.append(np.mean(e))
    f1d = np.array(f1d, dtype=float)
    f1d = np.expand_dims(f1d,0)
    f1d = self.scaler.transform(f1d)
    f1d = np.squeeze(f1d)
    f1d = torch.from_numpy(f1d).float()


    label = self.file_list[index].split("/")[-1].split(".")[-3]
    label = self.classes.index(label)

    return [f1d,stft], label


train_file_list, valid_file_list = make_filepath_list(data_dir)

print('学習データ数 : ', len(train_file_list))
print(train_file_list[0])

print('検証データ数 : ', len(valid_file_list))
print(valid_file_list[0])

q_classes = "Q1 Q2 Q3 Q4".split(" ")

train_dataset = musicDataset(
    file_list=train_file_list, classes=q_classes, phase='train'
)

valid_dataset = musicDataset(
    file_list=valid_file_list, classes=q_classes, phase='valid'
)


index = 0
print("Dataset1 shape:", train_dataset.__getitem__(index)[0][0].size())
print("Dataset2 shape:", train_dataset.__getitem__(index)[0][1].size())
print("Dataset label:", train_dataset.__getitem__(index)[1])

学習データ数 :  8960
/content/datas/Q1/Q1.MT0005083488_3.wav
検証データ数 :  2240
/content/datas/Q1/Q1.MT0032235381_4.wav
Dataset1 shape: torch.Size([26])
Dataset2 shape: torch.Size([1, 173, 257])
Dataset label: 0


# DataLoader

In [ ]:
# バッチサイズの指定
batch_size = 64

# DataLoaderを作成
train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)

valid_dataloader = data.DataLoader(
    valid_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

# pin_mem and num_workers are acceleration technique -> https://qiita.com/sugulu_Ogawa_ISID/items/62f5f7adee083d96a587#11-num_workers

# 辞書にまとめる
dataloaders_dict = {
    'train': train_dataloader, 
    'valid': valid_dataloader
}

# 動作確認
# イテレータに変換
batch_iterator = iter(dataloaders_dict['train'])

# 1番目の要素を取り出す
inputs, labels = next(batch_iterator)

print(inputs[0].size())
print(inputs[1].size())
print(labels)


torch.Size([64, 26])
torch.Size([64, 1, 173, 257])
tensor([0, 2, 2, 3, 1, 2, 3, 3, 3, 2, 0, 2, 0, 1, 1, 0, 0, 0, 2, 0, 0, 1, 1, 3,
        3, 0, 1, 0, 1, 2, 3, 1, 0, 2, 0, 2, 0, 0, 1, 3, 1, 3, 2, 2, 3, 0, 2, 3,
        1, 2, 2, 0, 2, 0, 2, 2, 3, 1, 1, 2, 3, 0, 1, 1])


# Optimizer, Criterion

In [ ]:
from torch import optim

model     = MultiModalNet(input_ch=1, num_classes=4, num_1d_features=train_dataset.__getitem__(index)[0][0].size(0)).to('cuda')
optimizer = optim.SGD(model.parameters(),lr=0.05)
criterion = nn.CrossEntropyLoss()

# Training

In [ ]:
from tqdm import tqdm
# エポック数
num_epochs = 200


for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')
    loss_hist = [[],[]]
    acc_hist = [[],[]]
    
    for phase in ['train', 'valid']:
        
        if phase == 'train':
            # モデルを訓練モードに設定
            model.train()
        else:
            # モデルを推論モードに設定
            model.eval()
        
        # 損失和
        epoch_loss = 0.0
        # 正解数
        epoch_corrects = 0
        
        # DataLoaderからデータをバッチごとに取り出す
        with tqdm(dataloaders_dict[phase],unit='batch',colour='green' if phase == 'train' else 'red') as pbar:
          pbar.set_description('['+phase+'] Epoch %d'% (epoch+1))
          for inputs, labels in pbar:
              inputs = [inputs[0].to('cuda') , inputs[1].to('cuda')]
              labels = labels.to('cuda')
              # optimizerの初期化
              optimizer.zero_grad()
            
              # 学習時のみ勾配を計算させる設定にする
              with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                
                  # 損失を計算
                  loss = criterion(outputs, labels)
                
                  # ラベルを予測
                  _, preds = torch.max(outputs, 1)
                
                  # 訓練時はバックプロパゲーション
                  if phase == 'train':
                    # 逆伝搬の計算
                    loss.backward()
                    # パラメータの更新
                    optimizer.step()
                
                  # イテレーション結果の計算
                  # lossの合計を更新
                  # PyTorchの仕様上各バッチ内での平均のlossが計算される。
                  # データ数を掛けることで平均から合計に変換をしている。
                  # 損失和は「全データの損失/データ数」で計算されるため、
                  # 平均のままだと損失和を求めることができないため。
                  l = loss.item()
                  epoch_loss += l * inputs[0].size(0)
                  pbar.set_postfix(dict(loss=l))

                  # 正解数の合計を更新
                  epoch_corrects += torch.sum(preds == labels.data)

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
        loss_hist[0].append(epoch_loss) if phase == 'train' else loss_hist[1].append(epoch_loss)
        acc_hist.append(epoch_acc)      if phase == 'train' else acc_hist[1].append(epoch_acc)

        print('[{}] Epoch {} Result: Loss: {:.4f} Acc: {:.4f}\n'.format(phase, epoch+1, epoch_loss, epoch_acc))

Epoch 1/200
-------------


[train] Epoch 1:   9%|▊         | 12/140 [01:25<11:53,  5.57s/batch, loss=1.39]/usr/local/lib/python3.7/dist-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
[train] Epoch 1: 100%|██████████| 140/140 [14:45<00:00,  6.33s/batch, loss=1.25]


[train] Epoch 1 Result: Loss: 1.3197 Acc: 0.3602



[valid] Epoch 1: 100%|██████████| 35/35 [03:34<00:00,  6.12s/batch, loss=1.2]


[valid] Epoch 1 Result: Loss: 1.2470 Acc: 0.4705

Epoch 2/200
-------------


...

[train] Epoch 37 Result: Loss: 0.3091 Acc: 0.8799



[valid] Epoch 37: 100%|██████████| 35/35 [03:33<00:00,  6.11s/batch, loss=0.581]


[valid] Epoch 37 Result: Loss: 0.7193 Acc: 0.7491

Epoch 38/200
-------------


[train] Epoch 38:  10%|█         | 14/140 [01:28<10:49,  5.16s/batch, loss=0.396]/usr/local/lib/python3.7/dist-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
[train] Epoch 38:  64%|██████▍   | 90/140 [09:27<05:15,  6.31s/batch, loss=0.233]


KeyboardInterrupt: ignored

# Visualization

In [ ]:
from matplotlib import pyplot as plt
# 学習状況のプロット関数の定義
# 分類精度の履歴をプロット
plt.plot(acc_hist[0])
plt.plot(acc_hist[1])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['training', 'validation'], loc='lower right')
plt.show()

# 損失関数の履歴をプロット
plt.plot(loss_hist[0])
plt.plot(loss_hist[1])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()


# 疑問点
入力画素数はどこでわかる？